# 实验1：熵的计算

## 1.1 创建数据集

In [1]:
"""
函数说明:创建数据集

Parameters:
	无
Returns:
	dataSet - 数据集
	attributes - 特征属性
"""
def createDataSet():
	dataSet = [[0, 0, 0, 0, 'no'],						#数据集
			[0, 0, 0, 1, 'no'],
			[0, 1, 0, 1, 'yes'],
			[0, 1, 1, 0, 'yes'],
			[0, 0, 0, 0, 'no'],
			[1, 0, 0, 0, 'no'],
			[1, 0, 0, 1, 'no'],
			[1, 1, 1, 1, 'yes'],
			[1, 0, 1, 2, 'yes'],
			[1, 0, 1, 2, 'yes'],
			[2, 0, 1, 2, 'yes'],
			[2, 0, 1, 1, 'yes'],
			[2, 1, 0, 1, 'yes'],
			[2, 1, 0, 2, 'yes'],
			[2, 0, 0, 0, 'no']]
	attributes = ['年龄', '有工作', '有自己的房子', '信贷情况']		#特征标签
	return dataSet, attributes 							#返回数据集和分类属性

# 测试熵函数calcShannonEnt
dataSet, attributes = createDataSet()
print(dataSet)

[[0, 0, 0, 0, 'no'], [0, 0, 0, 1, 'no'], [0, 1, 0, 1, 'yes'], [0, 1, 1, 0, 'yes'], [0, 0, 0, 0, 'no'], [1, 0, 0, 0, 'no'], [1, 0, 0, 1, 'no'], [1, 1, 1, 1, 'yes'], [1, 0, 1, 2, 'yes'], [1, 0, 1, 2, 'yes'], [2, 0, 1, 2, 'yes'], [2, 0, 1, 1, 'yes'], [2, 1, 0, 1, 'yes'], [2, 1, 0, 2, 'yes'], [2, 0, 0, 0, 'no']]


## 1.2 熵的计算

In [2]:
from math import log
"""
函数说明:计算给定数据集的经验熵(香农熵)

Parameters:
	dataSet - 数据集
Returns:
	shannonEnt - 经验熵(香农熵)
Author:
	Jack Cui
Blog:
	http://blog.csdn.net/c406495762
Modify:
	2017-07-24
"""
def calcShannonEnt(dataSet):
	numEntires = len(dataSet)						#返回数据集的行数
	labelCounts = {}								#保存每个标签(Label)出现次数的字典
	for featVec in dataSet:							#对每组特征向量进行统计
		currentLabel = featVec[-1]					#提取标签(Label)信息
		if currentLabel not in labelCounts.keys():	#如果标签(Label)没有放入统计次数的字典,添加进去
			labelCounts[currentLabel] = 0
		labelCounts[currentLabel] += 1				#Label计数
	shannonEnt = 0.0								#经验熵(香农熵)
	for key in labelCounts:							#计算香农熵
		prob = float(labelCounts[key]) / numEntires	#选择该标签(Label)的概率
		shannonEnt -= prob * log(prob, 2)			#利用公式计算
	return shannonEnt								#返回经验熵(香农熵)

In [3]:
print(calcShannonEnt(dataSet))

0.9709505944546686


# 实验2：条件熵的计算

In [4]:
"""
函数说明:按照给定属性划分数据集

Parameters:
	dataSet - 待划分的数据集
	axis - 划分数据集的属性索引
	value - 需要返回的属性的值
Returns:
	无
"""
def splitDataSet(dataSet, axis, value):		
	retSubDataSet = []										#创建返回的数据集列表
	for featVec in dataSet: 							#遍历数据集
		if featVec[axis] == value:
			reducedFeatVec = featVec[:axis]				#去掉axis特征
			reducedFeatVec.extend(featVec[axis+1:]) 	#将符合条件的添加到返回的数据集
			retSubDataSet.append(reducedFeatVec)
	return retSubDataSet		  							#返回划分后的数据集

In [5]:
"""
函数说明:计算给定数据集的某个属性下的条件熵

Parameters:
	dataSet - 数据集
	axis - 划分数据集的属性索引
Returns:
	conditionEntropy - 某个属性下的条件熵的值
"""
def calcCondShannonEnt(dataSet, axis):
	valueList = [example[axis] for example in dataSet]
	uniqueVals = set(valueList)     					#创建set集合{},元素不可重复
	conditionEntropy = 0.0  								#经验条件熵
	for value in uniqueVals: 						#计算信息增益
		subDataSet = splitDataSet(dataSet, axis, value) 		#subDataSet划分后的子集
		prob = len(subDataSet) / float(len(dataSet))   		#计算子集的概率
		conditionEntropy += prob * calcShannonEnt(subDataSet) 	#根据公式计算经验条件熵
	return conditionEntropy

In [6]:
# 测试条件熵函数calcCondShannonEnt
for i in range(len(attributes)):
	print("Attribute:{}".format(attributes[i]))
	print("Condition Entropy:{}".format(calcCondShannonEnt(dataSet, i)))

Attribute:年龄
Condition Entropy:0.8879430945988998
Attribute:有工作
Condition Entropy:0.6473003963031123
Attribute:有自己的房子
Condition Entropy:0.5509775004326937
Attribute:信贷情况
Condition Entropy:0.6079610319175832


# 实验3：基于信息增益的特征选择

In [7]:
"""
函数说明:基于信息增益选择最优特征

Parameters:
	dataSet - 数据集
Returns:
	bestAttribute - 信息增益最大的(最优)特征的索引值
"""
def chooseBestAttributeByInfoGain(dataSet):
	numAttributes = len(dataSet[0]) - 1					#属性数量
	baseEntropy = calcShannonEnt(dataSet) 				#计算数据集的香农熵
	bestInfoGain = 0.0  								#信息增益
	bestFeature = -1									#最优特征的索引值
	for i in range(numAttributes): 						#遍历所有特征
		conditionEntropy = calcCondShannonEnt(dataSet, i)
		infoGain = baseEntropy - conditionEntropy 					#信息增益
		print("第{}个特征的信息增益为{:.3f}".format(i, infoGain))			#打印每个特征的信息增益
		if (infoGain > bestInfoGain): 							#计算信息增益
			bestInfoGain = infoGain 							#更新信息增益，找到最大的信息增益
			bestAttribute = i 									#记录信息增益最大的特征的索引值
	return bestAttribute, bestInfoGain											#返回信息增益最大的特征的索引值

In [8]:
# 测试条件熵函数calcCondShannonEnt
bestAttribute, bestInfoGain = chooseBestAttributeByInfoGain(dataSet)
print("基于信息增益选择最优特征为第{}个特征:{}".format(bestAttribute, attributes[bestAttribute]))

第0个特征的信息增益为0.083
第1个特征的信息增益为0.324
第2个特征的信息增益为0.420
第3个特征的信息增益为0.363
基于信息增益选择最优特征为第2个特征:有自己的房子


# 实验4：基于ID3算法的决策树生成

In [9]:
"""
函数说明:统计classList中出现此处最多的元素(类标签)

Parameters:
	classList - 类标签列表
Returns:
	sortedClassCount[0][0] - 出现此处最多的元素(类标签)
Author:
	Jack Cui
Blog:
	http://blog.csdn.net/c406495762
Modify:
	2017-07-24
"""
def majorityCnt(classList):
	classCount = {}
	for vote in classList:										#统计classList中每个元素出现的次数
		if vote not in classCount.keys():classCount[vote] = 0	
		classCount[vote] += 1
	sortedClassCount = sorted(classCount.items(), key = lambda k:k[1], reverse = True)		#根据字典的值降序排序
	return sortedClassCount[0][0]								#返回classList中出现次数最多的元素


In [10]:
"""
函数说明:创建决策树

Parameters:
	dataSet - 训练数据集
	attributes - 特征属性集
	threshold - 阈值
Returns:
	myTree - 决策树
"""
def createTree(dataSet, attributes, threshold):
	classList = [example[-1] for example in dataSet]			#取分类标签(是否放贷:yes or no)
	if classList.count(classList[0]) == len(classList):			#算法5.2（1），如果类别完全相同则停止继续划分
		return classList[0]
	if len(attributes) == 0:									#算法5.2（2），遍历完所有特征时返回出现次数最多的类标签
		return majorityCnt(classList)
	bestAttribute, bestInfoGain = chooseBestAttributeByInfoGain(dataSet)		#算法5.2（3）选择最优特征
	print("属性集:{},最优属性:{},信息增益:{:.3f}".format(attributes, bestAttribute, bestInfoGain))
	bestAttributeLabel = attributes[bestAttribute]							#最优特征的标签
	if bestInfoGain < threshold:								#算法5.2（4），如果信息增益小于阈值则停止继续划分
		return majorityCnt(classList)
	myTree = {bestAttributeLabel:{}}									#根据最优特征的标签生成树
	del(attributes[bestAttribute])								#算法5.2（5，6），删除已经使用特征标签
	attributeValues = [example[bestAttribute] for example in dataSet]		#得到训练集中所有最优特征的属性值
	uniqueVals = set(attributeValues)							#去掉重复的属性值
	for value in uniqueVals:									#遍历特征，创建决策树。
		subAttributes = attributes[:]
		myTree[bestAttributeLabel][value] = createTree(splitDataSet(dataSet, bestAttribute, value), subAttributes, threshold)
        
	return myTree

In [11]:
threshold = 0.01
dataSet, attributes = createDataSet()
myTree = createTree(dataSet, attributes, threshold)
print(myTree)

第0个特征的信息增益为0.083
第1个特征的信息增益为0.324
第2个特征的信息增益为0.420
第3个特征的信息增益为0.363
属性集:['年龄', '有工作', '有自己的房子', '信贷情况'],最优属性:2,信息增益:0.420
第0个特征的信息增益为0.252
第1个特征的信息增益为0.918
第2个特征的信息增益为0.474
属性集:['年龄', '有工作', '信贷情况'],最优属性:1,信息增益:0.918
{'有自己的房子': {0: {'有工作': {0: 'no', 1: 'yes'}}, 1: 'yes'}}


# 实验5：基于决策树的分类

In [12]:
next(iter(myTree))

'有自己的房子'

In [13]:
"""
函数说明:使用决策树分类
 
Parameters:
    inputTree - 已经生成的决策树
    attributes - 属性标签
    testVec - 测试数据列表
Returns:
    classLabel - 分类结果
"""
def classify(inputTree, attributes, testVec):
    firstStr = next(iter(inputTree))                                                        #获取决策树结点
    secondDict = inputTree[firstStr]                                                        #下一个字典
    attributeIndex = attributes.index(firstStr)                                               
    for key in secondDict.keys():
        if testVec[attributeIndex] == key:
            if type(secondDict[key]).__name__ == 'dict':
                classLabel = classify(secondDict[key], attributes, testVec)
            else: classLabel = secondDict[key]
    return classLabel

In [14]:
threshold = 0.01
dataSet, attributes = createDataSet()
attributeLabels = []
attributeLabels.extend(attributes)
myTree = createTree(dataSet, attributes, threshold)
print(myTree)
print(attributeLabels)
testVec = [0,0,0,1]                                        #测试数据
result = classify(myTree, attributeLabels, testVec)
print(result)
testVec = [1,1,1,1]                                        #测试数据
result = classify(myTree, attributeLabels, testVec)
print(result)

第0个特征的信息增益为0.083
第1个特征的信息增益为0.324
第2个特征的信息增益为0.420
第3个特征的信息增益为0.363
属性集:['年龄', '有工作', '有自己的房子', '信贷情况'],最优属性:2,信息增益:0.420
第0个特征的信息增益为0.252
第1个特征的信息增益为0.918
第2个特征的信息增益为0.474
属性集:['年龄', '有工作', '信贷情况'],最优属性:1,信息增益:0.918
{'有自己的房子': {0: {'有工作': {0: 'no', 1: 'yes'}}, 1: 'yes'}}
['年龄', '有工作', '有自己的房子', '信贷情况']
no
yes
